## 1장 철학과 개념
### boolen 자료형을 비교할 땐 == 연산자를 사용하지마라

In [11]:
isPrime=True
if isPrime: print("1. no operation")
if isPrime==True: print("2. == operation")
if isPrime is True: print("3. 'is' operation")

1. no operation
2. == operation
3. 'is' operation



boolean 자료형은 연산자를 통한 비교 대신 그자체로 쓰이는게 맞음 따라서 2번은 좋은 코드가 아니다.
3번을 사용하면 안되는이유는 현재 우리는 동등성을 확인하는거지 
동일성을 확인하는 것이 아니기 때문에 이상황에서 is를 사용하는 것은 옳지 않다.


### 변수의 유효 범위 scope

* 파이썬의 유효범위를 계산할 때 네임스페이스를 기반으로 계산함
  * 어떤 변수가 사용됐다면 네임스페이스를 확인해 사용된 변수가 네임스페이스 안에 있는지 확인하고 없으면 nameError를 이르킨다.

* namespace (legb)
 * built-in   : 파이썬 내장 네임스페이스로 어디서든 사용가능
 * gobal      : 전역적으로 사용이 가능한 네임스페이스로 파일단위의 모듈에서 유효함 또 import하면 그 모듈의 global 변수도 사용가능
 * enclosed   : nested function에서 내부함수가 외부함수의 변수를 접근가능한 것 
 * local      : 함수나 클래스 내부로 한정

* LEGB: 변수를 확인할때 유효범위를 확안하는 순서

#### global
* first 정상 작동이유
  * 전역변수 msg를 출력하기 때문이다.
 * second 오류 발생 이유
  * msg 확인시 local에서 먼저 찾음    
  => msg="wow 존재 따라서 로컬변수     
  => msg="wow" 이 코드는 전역변수를 수정하는 것이 아닌 지역변수를 선언하는 코드이다      
  => 따라서 print(msg)는 선언전에 사용되었으므로 에러가 발생한다.

In [18]:
msg="hello"

def first()->None:
    print(msg)
def first_two()->None:
    global msg
    print(msg)
    msg="wow"
def second()->None:
    print(msg)
    msg="wow"
first()
first_two()
second()

hello
hello


UnboundLocalError: local variable 'msg' referenced before assignment

In [21]:
msg="바뀔거같냐? 지역변수 선언이지 바꾸는거 아님 legb 기억하셈 바꾸고 싶으면 global keyword쓰셈"
def modifyGlobal():
    msg="a"
modifyGlobal()
print(msg)

바뀔거같냐? 지역변수 선언이지 바꾸는거 아님 legb 기억하셈 바꾸고 싶으면 global keyword쓰셈


### nonlocal
* 두번째 예제에서 count가 증가하지 않는이유
* 1. legb 순서로 count를 찾는다
* 2. local에서 count를 찾음 근데 count+=1하고 있기때문에 선언 전에 사용한것 따라서 에러남
* 따라서 로컬에서 count를 찾으면 안돼기 때문에 nonlocal로 해준다.

In [26]:
def solveProblem():
    count=10
    def countReader():
        print(count)
    countReader()
    return count
solveProblem()

10


10

In [27]:
def solveProblem():
    count=10
    def counter():
        count+=1
    counter()
    return count
solveProblem()

UnboundLocalError: local variable 'count' referenced before assignment

In [25]:
def solveProblem():
    count=10
    def counter():
        nonlocal count
        count+=1
    counter()
    return count
solveProblem()


11

### first class citizen
* 일급 객체
  * 어떤 개체를 다른개체의 매개변수로 전달
  * 함수의 반환값으로 사용하거나 변수의 값을 할당할 수 있는것  

### first class function 
 * 함수가 first class citizen 속성을 가지는 것

### higher order function(고차 함수)

* 둘중 하나만 만족하면 고차함수임
  * 함수가 매개변수로 전달
  * 함수를 반환값으로 사용


In [3]:
def square(x:int)->int:
    return x*x
f=square
f(10)
def funcBinder(func,arg_list):
    return [func(i) for i in arg_list]
answer=funcBinder(f,[10,11,12])
print(answer)

[100, 121, 144]


In [4]:
def UpperMaker(data):
    return data.upper()
list(map(UpperMaker,["fwesf","fergfb","tyhty"]))

['FWESF', 'FERGFB', 'TYHTY']

### Nested function
* 함수로 감싸인 함수
* 외부함수와 내부함수로 이루어짐
* 각각의 내부함수들은 scope chain에 의해 자신을 감싸고 있는 외부함수의 메모리에 접근이 가능하다

In [1]:
def calculator(x:int,y:int):
    def add(): return x+y
    def sub(): return x-y
    return (add(),sub())
print(calculator(3,4))

(7, -1)


In [9]:
def multiple(x:int):
    def mul(y):
        return x**y
    return mul
baseFive=multiple(5)
baseTen=multiple(10)

print(baseFive(1))
print(baseTen(2))

5
100


### closure
* 아래와 같은 예제를 closure라고 한다
* 함수의 반환값으로 내부 함수를 사용하는 함수
* 아래예제에서 base를 접근 할 수있는 이유? (이미 addFunc은 종료되었기 때문에 base역시 접근 불가할것 같은데?)  
=> 클로져는 내부 함수를 반환하지만 이때 이 함수와 관련된 외부 환경은 모두 저장함
 * 어디다 저장? \__closure\__

In [21]:
def addFunc(x,y):
    base1=x
    base2=y
    def add(z):
        return base1+base2+z
    return add
f=addFunc(2,3)
print(f(10))

print(f.__closure__[0].cell_contents)
print(f.__closure__[1].cell_contents)

15
2
3


In [13]:
def multiple(x:int):
    base=x
    def mul(y):
        return base**y
    return mul
baseFive=multiple(5)
baseTen=multiple(10)

print(baseFive(1))
print(baseTen(2))

print(baseFive.__closure__[0].cell_contents)

5
100
5


### \*args와 \*\*kwargs
* 가변 매개변수임
* \*args는 non-keyworded 가변인자
  * list ...
* \*\*kwargs는 keyworded 가변인자임
  * dict ...

### Partial Application
* 복수의 매개변수를 갖는 함수에서 일부 매개변수의 값이 고정적일때 매번 고정된 매개변수를 작성해서 함수를 호출 하는 것은 비효율적임
  * 매개변수의 일부를 미리 전달해서(지역 변수화 시켜버린다.) wrapping 함수를 만들고 이 래핑된 함수를 사용해 가변적인 매개변수만 매개변수로 사용하는 기법임
  * 구현 하는 방법
    * closure를 활용
    * from functools import partial 

In [12]:
#구현 1
def stringPrinter(*args):
    for i in args:
        print(i,end=" ")
    print()
def partial(func,*partial_args):
    
    def wrapper(*extra_args):
        args=list(partial_args)
        args.extend(list(extra_args))
        return func(*args)
    return wrapper
    
stringPrinter("hi","my","name","is","yoonseok")
'''
hi my name is yoonseok > 고정으로 사용될 인자!
'''
f=partial(stringPrinter,"hi","my","name","is","yoonseok")
f("so","what?")
f("he he")

hi my name is yoonseok 
hi my name is yoonseok so what? 
hi my name is yoonseok he he 


In [24]:
#구현 2
from  functools import partial
f=partial(stringPrinter,"bear","robotics","go")
f("i'm","gonna","miss",'you',"and","korea")

def listPrinter(num1:list,num2:list)->None:
    
    print(f"start:{num1}")
    print(f"end: {num2}")
    
f=partial(listPrinter,["bear","robotics","go"])
f(["i'm","gonna","miss",'you',"and","korea"])


bear robotics go i'm gonna miss you and korea 
start:['bear', 'robotics', 'go']
end: ["i'm", 'gonna', 'miss', 'you', 'and', 'korea']
